## Creating service with generalised feature classes for admins

In the country level give a size category to each country. Then use that field in the admin1 level to create the different feature collections per zoom level. 
Once the level 1 have been divided, create a new field in the country that is one more value than the previously calculated.

- Living atlas [service](https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/World_Administrative_Divisions/FeatureServer) with level 1 admin borders
- Generalised [service](https://services9.arcgis.com/IkktFdUAcY3WrH25/arcgis/rest/services/gadm_generalised/FeatureServer) with the country borders used in the National Report Cards (missing Fiji and Kiribati)
- Fiji and Kiribati [service](https://services9.arcgis.com/IkktFdUAcY3WrH25/arcgis/rest/services/KIR_FIJ_proj4716/FeatureServer)

In [ ]:
arcpy.analysis.Select("World_Administrative_Divisions", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\World_Administrative_Divisio", "DISPUTED <= 1")

In [ ]:
with arcpy.EnvManager(outputCoordinateSystem="GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]"):
    arcpy.management.Merge("KIR_FIJ_proj4716;LA_GADM_clean_dateline", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\generalised_nrc", 'GID_0 "GID_0" true true false 8000 Text 0 0,First,#,KIR_FIJ_proj4716,GID_0,0,8000,LA_GADM_clean_dateline,GID_0,0,8000;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,KIR_FIJ_proj4716,NAME_0,0,8000,LA_GADM_clean_dateline,NAME_0,0,8000;Shape__Area "Shape__Area" false true false 0 Double 0 0,First,#,KIR_FIJ_proj4716,Shape__Area,-1,-1,LA_GADM_clean_dateline,Shape__Area,-1,-1;Shape__Length "Shape__Length" false true false 0 Double 0 0,First,#,KIR_FIJ_proj4716,Shape__Length,-1,-1,LA_GADM_clean_dateline,Shape__Length,-1,-1', "NO_SOURCE_INFO")

In [ ]:
arcpy.management.CalculateField("generalised_nrc", "country_size", "country_size(!Shape_Area!)", "PYTHON3", r"def country_size(poly_area):
    if poly_area <= 10:
        a = 5
    if poly_area>10 and poly_area<=30:
        a = 4
    if poly_area>30 and poly_area<=100:
        a =3
    if poly_area>100 and poly_area<=952:
        a =2
    if poly_area>952 and poly_area<=6053:
         a =1
    return a", "LONG")

In [ ]:
arcpy.management.JoinField("generalised_nrc", "GID_0", "countrycode.csv", "iso3", "iso2")

In [ ]:
arcpy.management.JoinField("World_Administrative_Divisio", "ISO_CC", "generalised_nrc", "iso2", "GID_0;country_size")

In [ ]:
for zl in [1,2,3,4,5]:
    arcpy.analysis.Select("World_Administrative_Divisio", 
                          f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/World_Administrative_Divisio_zoom_{zl}", f"country_size = {zl}")

In [ ]:
for zl in [0,1,2,3,4]:
    arcpy.analysis.Select("generalised_nrc", 
                          f"C:/Users/Viz1/Documents/ArcGIS/Projects/SuperSample/SuperSample.gdb/generalised_nrc_zoom{zl}", f"country_zoom = {zl}")

### Creating a service per zoom level. 

In [16]:
arcpy.management.Merge(r"zoom4\World_Administrative_Divisio_zoom_4;zoom4\generalised_nrc_zoom4;zoom4\World_Administrative_Divisio_zoom_3;zoom4\World_Administrative_Divisio_zoom_2;zoom4\World_Administrative_Divisio_zoom_1", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\zoom4_Merge", r'NAME "Name" true true false 44 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,NAME,0,44,zoom4\World_Administrative_Divisio_zoom_3,NAME,0,44,zoom4\World_Administrative_Divisio_zoom_2,NAME,0,44,zoom4\World_Administrative_Divisio_zoom_1,NAME,0,44;COUNTRY "Country" true true false 50 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,COUNTRY,0,50,zoom4\World_Administrative_Divisio_zoom_3,COUNTRY,0,50,zoom4\World_Administrative_Divisio_zoom_2,COUNTRY,0,50,zoom4\World_Administrative_Divisio_zoom_1,COUNTRY,0,50;ISO_CODE "ISO Code" true true false 6 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,ISO_CODE,0,6,zoom4\World_Administrative_Divisio_zoom_3,ISO_CODE,0,6,zoom4\World_Administrative_Divisio_zoom_2,ISO_CODE,0,6,zoom4\World_Administrative_Divisio_zoom_1,ISO_CODE,0,6;ISO_CC "ISO Country" true true false 2 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,ISO_CC,0,2,zoom4\World_Administrative_Divisio_zoom_3,ISO_CC,0,2,zoom4\World_Administrative_Divisio_zoom_2,ISO_CC,0,2,zoom4\World_Administrative_Divisio_zoom_1,ISO_CC,0,2;ISO_SUB "ISO Admin" true true false 5 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,ISO_SUB,0,5,zoom4\World_Administrative_Divisio_zoom_3,ISO_SUB,0,5,zoom4\World_Administrative_Divisio_zoom_2,ISO_SUB,0,5,zoom4\World_Administrative_Divisio_zoom_1,ISO_SUB,0,5;ADMINTYPE "Admin Type" true true false 50 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,ADMINTYPE,0,50,zoom4\World_Administrative_Divisio_zoom_3,ADMINTYPE,0,50,zoom4\World_Administrative_Divisio_zoom_2,ADMINTYPE,0,50,zoom4\World_Administrative_Divisio_zoom_1,ADMINTYPE,0,50;DISPUTED "Disputed" true true false 4 Long 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,DISPUTED,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,DISPUTED,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,DISPUTED,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,DISPUTED,-1,-1;NOTES "Notes" true true false 254 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,NOTES,0,254,zoom4\World_Administrative_Divisio_zoom_3,NOTES,0,254,zoom4\World_Administrative_Divisio_zoom_2,NOTES,0,254,zoom4\World_Administrative_Divisio_zoom_1,NOTES,0,254;AUTONOMOUS "Autonomous" true true false 4 Long 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,AUTONOMOUS,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,AUTONOMOUS,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,AUTONOMOUS,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,AUTONOMOUS,-1,-1;COUNTRYAFF "Affiliation" true true false 100 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,COUNTRYAFF,0,100,zoom4\World_Administrative_Divisio_zoom_3,COUNTRYAFF,0,100,zoom4\World_Administrative_Divisio_zoom_2,COUNTRYAFF,0,100,zoom4\World_Administrative_Divisio_zoom_1,COUNTRYAFF,0,100;CONTINENT "Continent" true true false 13 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,CONTINENT,0,13,zoom4\World_Administrative_Divisio_zoom_3,CONTINENT,0,13,zoom4\World_Administrative_Divisio_zoom_2,CONTINENT,0,13,zoom4\World_Administrative_Divisio_zoom_1,CONTINENT,0,13;LAND_TYPE "Land_Type" true true false 20 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,LAND_TYPE,0,20,zoom4\World_Administrative_Divisio_zoom_3,LAND_TYPE,0,20,zoom4\World_Administrative_Divisio_zoom_2,LAND_TYPE,0,20,zoom4\World_Administrative_Divisio_zoom_1,LAND_TYPE,0,20;LAND_RANK "Land_Rank" true true false 4 Long 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,LAND_RANK,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,LAND_RANK,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,LAND_RANK,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,LAND_RANK,-1,-1;GID_0 "GID_0" true true false 8000 Text 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,GID_0,0,8000,zoom4\generalised_nrc_zoom4,GID_0,0,8000,zoom4\World_Administrative_Divisio_zoom_3,GID_0,0,8000,zoom4\World_Administrative_Divisio_zoom_2,GID_0,0,8000,zoom4\World_Administrative_Divisio_zoom_1,GID_0,0,8000;country_size "country_size" true true false 4 Long 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,country_size,-1,-1,zoom4\generalised_nrc_zoom4,country_size,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,country_size,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,country_size,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,country_size,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,Shape_Length,-1,-1,zoom4\generalised_nrc_zoom4,Shape_Length,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,Shape_Length,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,Shape_Length,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,zoom4\World_Administrative_Divisio_zoom_4,Shape_Area,-1,-1,zoom4\generalised_nrc_zoom4,Shape_Area,-1,-1,zoom4\World_Administrative_Divisio_zoom_3,Shape_Area,-1,-1,zoom4\World_Administrative_Divisio_zoom_2,Shape_Area,-1,-1,zoom4\World_Administrative_Divisio_zoom_1,Shape_Area,-1,-1;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,zoom4\generalised_nrc_zoom4,NAME_0,0,8000;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,zoom4\generalised_nrc_zoom4,country_zoom,-1,-1;iso2 "iso2" true true false 8000 Text 0 0,First,#,zoom4\generalised_nrc_zoom4,iso2,0,8000', "NO_SOURCE_INFO")
arcpy.management.Merge(r"zoom3\World_Administrative_Divisio_zoom_1;zoom3\World_Administrative_Divisio_zoom_2;zoom3\generalised_nrc_zoom3;zoom3\World_Administrative_Divisio_zoom_3;zoom3\generalised_nrc_zoom4", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\zoom3_Merge", r'NAME "Name" true true false 44 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,NAME,0,44,zoom3\World_Administrative_Divisio_zoom_2,NAME,0,44,zoom3\World_Administrative_Divisio_zoom_3,NAME,0,44;COUNTRY "Country" true true false 50 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,COUNTRY,0,50,zoom3\World_Administrative_Divisio_zoom_2,COUNTRY,0,50,zoom3\World_Administrative_Divisio_zoom_3,COUNTRY,0,50;ISO_CODE "ISO Code" true true false 6 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,ISO_CODE,0,6,zoom3\World_Administrative_Divisio_zoom_2,ISO_CODE,0,6,zoom3\World_Administrative_Divisio_zoom_3,ISO_CODE,0,6;ISO_CC "ISO Country" true true false 2 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,ISO_CC,0,2,zoom3\World_Administrative_Divisio_zoom_2,ISO_CC,0,2,zoom3\World_Administrative_Divisio_zoom_3,ISO_CC,0,2;ISO_SUB "ISO Admin" true true false 5 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,ISO_SUB,0,5,zoom3\World_Administrative_Divisio_zoom_2,ISO_SUB,0,5,zoom3\World_Administrative_Divisio_zoom_3,ISO_SUB,0,5;ADMINTYPE "Admin Type" true true false 50 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,ADMINTYPE,0,50,zoom3\World_Administrative_Divisio_zoom_2,ADMINTYPE,0,50,zoom3\World_Administrative_Divisio_zoom_3,ADMINTYPE,0,50;DISPUTED "Disputed" true true false 4 Long 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,DISPUTED,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,DISPUTED,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,DISPUTED,-1,-1;NOTES "Notes" true true false 254 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,NOTES,0,254,zoom3\World_Administrative_Divisio_zoom_2,NOTES,0,254,zoom3\World_Administrative_Divisio_zoom_3,NOTES,0,254;AUTONOMOUS "Autonomous" true true false 4 Long 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,AUTONOMOUS,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,AUTONOMOUS,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,AUTONOMOUS,-1,-1;COUNTRYAFF "Affiliation" true true false 100 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,COUNTRYAFF,0,100,zoom3\World_Administrative_Divisio_zoom_2,COUNTRYAFF,0,100,zoom3\World_Administrative_Divisio_zoom_3,COUNTRYAFF,0,100;CONTINENT "Continent" true true false 13 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,CONTINENT,0,13,zoom3\World_Administrative_Divisio_zoom_2,CONTINENT,0,13,zoom3\World_Administrative_Divisio_zoom_3,CONTINENT,0,13;LAND_TYPE "Land_Type" true true false 20 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,LAND_TYPE,0,20,zoom3\World_Administrative_Divisio_zoom_2,LAND_TYPE,0,20,zoom3\World_Administrative_Divisio_zoom_3,LAND_TYPE,0,20;LAND_RANK "Land_Rank" true true false 4 Long 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,LAND_RANK,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,LAND_RANK,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,LAND_RANK,-1,-1;GID_0 "GID_0" true true false 8000 Text 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,GID_0,0,8000,zoom3\World_Administrative_Divisio_zoom_2,GID_0,0,8000,zoom3\generalised_nrc_zoom3,GID_0,0,8000,zoom3\World_Administrative_Divisio_zoom_3,GID_0,0,8000,zoom3\generalised_nrc_zoom4,GID_0,0,8000;country_size "country_size" true true false 4 Long 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,country_size,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,country_size,-1,-1,zoom3\generalised_nrc_zoom3,country_size,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,country_size,-1,-1,zoom3\generalised_nrc_zoom4,country_size,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,Shape_Length,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,Shape_Length,-1,-1,zoom3\generalised_nrc_zoom3,Shape_Length,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,Shape_Length,-1,-1,zoom3\generalised_nrc_zoom4,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,zoom3\World_Administrative_Divisio_zoom_1,Shape_Area,-1,-1,zoom3\World_Administrative_Divisio_zoom_2,Shape_Area,-1,-1,zoom3\generalised_nrc_zoom3,Shape_Area,-1,-1,zoom3\World_Administrative_Divisio_zoom_3,Shape_Area,-1,-1,zoom3\generalised_nrc_zoom4,Shape_Area,-1,-1;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,zoom3\generalised_nrc_zoom3,NAME_0,0,8000,zoom3\generalised_nrc_zoom4,NAME_0,0,8000;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,zoom3\generalised_nrc_zoom3,country_zoom,-1,-1,zoom3\generalised_nrc_zoom4,country_zoom,-1,-1;iso2 "iso2" true true false 8000 Text 0 0,First,#,zoom3\generalised_nrc_zoom3,iso2,0,8000,zoom3\generalised_nrc_zoom4,iso2,0,8000', "NO_SOURCE_INFO")
arcpy.management.Merge(r"zoom2\World_Administrative_Divisio_zoom_1;zoom2\generalised_nrc_zoom2;zoom2\World_Administrative_Divisio_zoom_2;zoom2\generalised_nrc_zoom3;zoom2\generalised_nrc_zoom4", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\zoom2_Merge", r'NAME "Name" true true false 44 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,NAME,0,44,zoom2\World_Administrative_Divisio_zoom_2,NAME,0,44;COUNTRY "Country" true true false 50 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,COUNTRY,0,50,zoom2\World_Administrative_Divisio_zoom_2,COUNTRY,0,50;ISO_CODE "ISO Code" true true false 6 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,ISO_CODE,0,6,zoom2\World_Administrative_Divisio_zoom_2,ISO_CODE,0,6;ISO_CC "ISO Country" true true false 2 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,ISO_CC,0,2,zoom2\World_Administrative_Divisio_zoom_2,ISO_CC,0,2;ISO_SUB "ISO Admin" true true false 5 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,ISO_SUB,0,5,zoom2\World_Administrative_Divisio_zoom_2,ISO_SUB,0,5;ADMINTYPE "Admin Type" true true false 50 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,ADMINTYPE,0,50,zoom2\World_Administrative_Divisio_zoom_2,ADMINTYPE,0,50;DISPUTED "Disputed" true true false 4 Long 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,DISPUTED,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,DISPUTED,-1,-1;NOTES "Notes" true true false 254 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,NOTES,0,254,zoom2\World_Administrative_Divisio_zoom_2,NOTES,0,254;AUTONOMOUS "Autonomous" true true false 4 Long 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,AUTONOMOUS,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,AUTONOMOUS,-1,-1;COUNTRYAFF "Affiliation" true true false 100 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,COUNTRYAFF,0,100,zoom2\World_Administrative_Divisio_zoom_2,COUNTRYAFF,0,100;CONTINENT "Continent" true true false 13 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,CONTINENT,0,13,zoom2\World_Administrative_Divisio_zoom_2,CONTINENT,0,13;LAND_TYPE "Land_Type" true true false 20 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,LAND_TYPE,0,20,zoom2\World_Administrative_Divisio_zoom_2,LAND_TYPE,0,20;LAND_RANK "Land_Rank" true true false 4 Long 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,LAND_RANK,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,LAND_RANK,-1,-1;GID_0 "GID_0" true true false 8000 Text 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,GID_0,0,8000,zoom2\generalised_nrc_zoom2,GID_0,0,8000,zoom2\World_Administrative_Divisio_zoom_2,GID_0,0,8000,zoom2\generalised_nrc_zoom3,GID_0,0,8000,zoom2\generalised_nrc_zoom4,GID_0,0,8000;country_size "country_size" true true false 4 Long 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,country_size,-1,-1,zoom2\generalised_nrc_zoom2,country_size,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,country_size,-1,-1,zoom2\generalised_nrc_zoom3,country_size,-1,-1,zoom2\generalised_nrc_zoom4,country_size,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,Shape_Length,-1,-1,zoom2\generalised_nrc_zoom2,Shape_Length,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,Shape_Length,-1,-1,zoom2\generalised_nrc_zoom3,Shape_Length,-1,-1,zoom2\generalised_nrc_zoom4,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,zoom2\World_Administrative_Divisio_zoom_1,Shape_Area,-1,-1,zoom2\generalised_nrc_zoom2,Shape_Area,-1,-1,zoom2\World_Administrative_Divisio_zoom_2,Shape_Area,-1,-1,zoom2\generalised_nrc_zoom3,Shape_Area,-1,-1,zoom2\generalised_nrc_zoom4,Shape_Area,-1,-1;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,zoom2\generalised_nrc_zoom2,NAME_0,0,8000,zoom2\generalised_nrc_zoom3,NAME_0,0,8000,zoom2\generalised_nrc_zoom4,NAME_0,0,8000;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,zoom2\generalised_nrc_zoom2,country_zoom,-1,-1,zoom2\generalised_nrc_zoom3,country_zoom,-1,-1,zoom2\generalised_nrc_zoom4,country_zoom,-1,-1;iso2 "iso2" true true false 8000 Text 0 0,First,#,zoom2\generalised_nrc_zoom2,iso2,0,8000,zoom2\generalised_nrc_zoom3,iso2,0,8000,zoom2\generalised_nrc_zoom4,iso2,0,8000', "NO_SOURCE_INFO")
arcpy.management.Merge(r"zoom1\generalised_nrc_zoom1;zoom1\World_Administrative_Divisio_zoom_1;zoom1\generalised_nrc_zoom2;zoom1\generalised_nrc_zoom3;zoom1\generalised_nrc_zoom4", r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\zoom1_Merge", r'GID_0 "GID_0" true true false 8000 Text 0 0,First,#,zoom1\generalised_nrc_zoom1,GID_0,0,8000,zoom1\World_Administrative_Divisio_zoom_1,GID_0,0,8000,zoom1\generalised_nrc_zoom2,GID_0,0,8000,zoom1\generalised_nrc_zoom3,GID_0,0,8000,zoom1\generalised_nrc_zoom4,GID_0,0,8000;NAME_0 "NAME_0" true true false 8000 Text 0 0,First,#,zoom1\generalised_nrc_zoom1,NAME_0,0,8000,zoom1\generalised_nrc_zoom2,NAME_0,0,8000,zoom1\generalised_nrc_zoom3,NAME_0,0,8000,zoom1\generalised_nrc_zoom4,NAME_0,0,8000;country_size "country_size" true true false 4 Long 0 0,First,#,zoom1\generalised_nrc_zoom1,country_size,-1,-1,zoom1\World_Administrative_Divisio_zoom_1,country_size,-1,-1,zoom1\generalised_nrc_zoom2,country_size,-1,-1,zoom1\generalised_nrc_zoom3,country_size,-1,-1,zoom1\generalised_nrc_zoom4,country_size,-1,-1;country_zoom "country_zoom" true true false 4 Long 0 0,First,#,zoom1\generalised_nrc_zoom1,country_zoom,-1,-1,zoom1\generalised_nrc_zoom2,country_zoom,-1,-1,zoom1\generalised_nrc_zoom3,country_zoom,-1,-1,zoom1\generalised_nrc_zoom4,country_zoom,-1,-1;iso2 "iso2" true true false 8000 Text 0 0,First,#,zoom1\generalised_nrc_zoom1,iso2,0,8000,zoom1\generalised_nrc_zoom2,iso2,0,8000,zoom1\generalised_nrc_zoom3,iso2,0,8000,zoom1\generalised_nrc_zoom4,iso2,0,8000;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,zoom1\generalised_nrc_zoom1,Shape_Length,-1,-1,zoom1\World_Administrative_Divisio_zoom_1,Shape_Length,-1,-1,zoom1\generalised_nrc_zoom2,Shape_Length,-1,-1,zoom1\generalised_nrc_zoom3,Shape_Length,-1,-1,zoom1\generalised_nrc_zoom4,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,zoom1\generalised_nrc_zoom1,Shape_Area,-1,-1,zoom1\World_Administrative_Divisio_zoom_1,Shape_Area,-1,-1,zoom1\generalised_nrc_zoom2,Shape_Area,-1,-1,zoom1\generalised_nrc_zoom3,Shape_Area,-1,-1,zoom1\generalised_nrc_zoom4,Shape_Area,-1,-1;NAME "Name" true true false 44 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,NAME,0,44;COUNTRY "Country" true true false 50 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,COUNTRY,0,50;ISO_CODE "ISO Code" true true false 6 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,ISO_CODE,0,6;ISO_CC "ISO Country" true true false 2 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,ISO_CC,0,2;ISO_SUB "ISO Admin" true true false 5 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,ISO_SUB,0,5;ADMINTYPE "Admin Type" true true false 50 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,ADMINTYPE,0,50;DISPUTED "Disputed" true true false 4 Long 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,DISPUTED,-1,-1;NOTES "Notes" true true false 254 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,NOTES,0,254;AUTONOMOUS "Autonomous" true true false 4 Long 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,AUTONOMOUS,-1,-1;COUNTRYAFF "Affiliation" true true false 100 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,COUNTRYAFF,0,100;CONTINENT "Continent" true true false 13 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,CONTINENT,0,13;LAND_TYPE "Land_Type" true true false 20 Text 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,LAND_TYPE,0,20;LAND_RANK "Land_Rank" true true false 4 Long 0 0,First,#,zoom1\World_Administrative_Divisio_zoom_1,LAND_RANK,-1,-1', "NO_SOURCE_INFO")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Datasets: Dataset zoom4\World_Administrative_Divisio_zoom_4 does not exist or is not supported
WARNING 000725: Output Dataset: Dataset C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\zoom4_Merge already exists.
Failed to execute (Merge).


## Generalising geometry of ecoregions
Information about the simplify tool and the different methods provided by the [documentation](https://desktop.arcgis.com/en/arcmap/latest/tools/cartography-toolbox/how-simplify-line-works.htm)

In [ ]:
arcpy.cartography.SimplifyPolygon("Ecoregions2017_20210825", 
                                  r"C:\Users\Viz1\Documents\ArcGIS\Projects\SuperSample\SuperSample.gdb\Ecoregions2017_simplify_100km", 
                                  "BEND_SIMPLIFY", "100 Kilometers", 
                                  "0 SquareMeters", "RESOLVE_ERRORS", "KEEP_COLLAPSED_POINTS", None)